## Краткая демонстрация к тестовому заданию

Для работы с базой данных нам понадобиться:
1. Сама база данных, поднимем бесплатную PostgreSQL на [filess.io](https://filess.io/).
2. Фреймворк [SQLAlchemy](https://www.sqlalchemy.org/).
3. Немного самодельных классов и функций.

Определим:
* `config.py` - параметры для установки соединения;
* `database.py` - параметры подключения к БД;
* `models.py` - классы и объекты с которыми мы будем работать (структуры таблиц);
* `core.py` - функции для работы в core (прямом) режиме;
* `orm.py` - функции для работы в ORM (объектном) режиме;
* `main.py` - непосредственная работа с базой данных;

Создаём и наполняем данными таблицы interns, grades и tools

In [35]:
from orm import SyncOrm

SyncOrm.create_tables()
SyncOrm.insert_interns()
SyncOrm.insert_grades()
SyncOrm.insert_tools()

Посмотрим, как составлять базовые запросы

Обратимся к базе данных и выведем всю таблицу **interns** экземплярами класса `InternsOrm`, который имеет следующие атрибуты:
```python
class InternsOrm(Base):
    __tablename__ = "interns"
    __table_args__ = {"schema": "cloveri"}
    id:         Mapped[str] = mapped_column(primary_key=True)
    email:      Mapped[str]
    name:       Mapped[str]
    role:       Mapped[str]
    internship: Mapped[str]
    start_date: Mapped[datetime.datetime]
    end_date:   Mapped[datetime.datetime]
    notes:      Mapped[str | None]
```

In [78]:
from sqlalchemy import select
from database import session_factory
from models import InternsOrm

with session_factory() as session:
    stmt = select(InternsOrm)
    print(f"{stmt.compile(compile_kwargs={"literal_binds": True})}\n")
    result = session.execute(stmt)
    interns = result.scalars().all()
    for model in interns:
        print(model.id, model.username, model.internship)

SELECT cloveri.interns.id, cloveri.interns.email, cloveri.interns.username, cloveri.interns.user_role, cloveri.interns.internship, cloveri.interns.start_date, cloveri.interns.end_date, cloveri.interns.notes 
FROM cloveri.interns

f1 Miguel Aubakirov Кабинет Кловери
f2 Miguel Aubakirov MAIBO
f3 Nombre Apellido Кабинет Кловери
f4 Jose Rodrigues MAIBO
f5 Soledad Lorna Платформа для ВУЗов


Простой запрос с фильтрацией, посмотрим сколько записей в таблице interns есть для пользователя с адресом электронной почты `work.aubakirov@gmail.com`

In [2]:
from sqlalchemy import select
from database import session_factory
from models import InternsOrm

intern_email = 'work.aubakirov@gmail.com'

with session_factory() as session:
    query = (
        select(InternsOrm)
        .filter(InternsOrm.email.contains(intern_email))
    )
    # выведем на экран сырой запрос, который отправится в базу
    print(f"{query.compile(compile_kwargs={"literal_binds": True})}\n")
    result = session.execute(query)
    interns = result.scalars().all()
    for model in interns:
        print(model.id, model.username, model.internship, model.user_role)

SELECT cloveri.interns.id, cloveri.interns.email, cloveri.interns.username, cloveri.interns.user_role, cloveri.interns.internship, cloveri.interns.start_date, cloveri.interns.end_date, cloveri.interns.notes 
FROM cloveri.interns 
WHERE (cloveri.interns.email LIKE '%' || 'work.aubakirov@gmail.com' || '%')

f1 Miguel Aubakirov Кабинет Кловери Python-программист
f2 Miguel Aubakirov MAIBO Python-программист


Посмотрим количество стажеров в каждом проекте

In [14]:
from sqlalchemy import select, func
from database import session_factory
from models import InternsOrm

with session_factory() as session:
    query = (
        select(InternsOrm.internship, func.count(InternsOrm.internship))
        .group_by(InternsOrm.internship)
        .order_by(InternsOrm.internship)
    )
    # выведем на экран сырой запрос, который отправится в базу
    print(f"{query.compile(compile_kwargs={"literal_binds": True})}\n")
    result = session.execute(query)
    interns = result.all()
    for model in interns:
        print(model)

SELECT cloveri.interns.internship, count(cloveri.interns.internship) AS count_1 
FROM cloveri.interns GROUP BY cloveri.interns.internship ORDER BY cloveri.interns.internship

('MAIBO', 2)
('Кабинет Кловери', 2)
('Платформа для ВУЗов', 1)


Объединим все таблицы в одну, получим их как объекты класса `InternsOrm` и `Pandas.DataFrame`.

In [80]:
from sqlalchemy import select, func
from sqlalchemy.orm import aliased
from database import session_factory
from models import InternsOrm, GradesOrm, ToolsOrm

import pandas as pd

i = aliased(InternsOrm, name="i")
g = aliased(GradesOrm, name="g")
t = aliased(ToolsOrm, name="t")

with session_factory() as session:
    stmt = (select(i, g, t)
            .join(i, i.id == g.intern_id)
            .join(t, i.user_role == t.user_role)
            .where(i.internship == t.internship))
    print(f"{stmt.compile(compile_kwargs={"literal_binds": True})}\n")
    result = session.execute(stmt)
    interns = result.scalars().all()
    for model in interns:
        print(model)
    # для удобства запишем результат запроса в pandas
    df = pd.read_sql(stmt, session.bind)

df.iloc[0].dropna()

SELECT i.id, i.email, i.username, i.user_role, i.internship, i.start_date, i.end_date, i.notes, g.intern_id, g.id AS id_1, g.ai, g.programming, g.architecture, g.project_management, g.communication, g.personnel_management, g.information_management, g.infrastructure, g.data_management, g.math, g.data_analysis, g.visual, g.ux_ui, g.seo, g.typografy, g.skill18, g.skill19, g.skill20, t.id AS id_2, t.user_role AS user_role_1, t.internship AS internship_1, t.pycaret, t.random_forest, t.anomaly_detection, t.gradient_boosting, t.cluster_analysis, t.k_fold, t.optuna, t.shap, t.tsfresh, t.sklearn_feature_selection, t.imblearn, t.streamlit, t.selenium, t.allure, t.nose, t.simpletest, t.docker, t.jira, t.adobe_xd, t.figma, t.mockplus, t.yandex_metrika, t.google_analytics, t.sketch, t.fontjoy, t.sqlalchemy, t.postgresql, t.git, t.django 
FROM cloveri.grades AS g JOIN cloveri.interns AS i ON i.id = g.intern_id JOIN cloveri.tools AS t ON i.user_role = t.user_role 
WHERE i.internship = t.internship



id                                              f1
email                     work.aubakirov@gmail.com
username                          Miguel Aubakirov
user_role                       Python-программист
internship                         Кабинет Кловери
start_date                     2025-01-15 00:00:00
end_date                       2025-04-28 00:00:00
intern_id                                       f1
id_1                                            g1
ai                                             4.4
programming                                    4.2
architecture                                   3.4
project_management                             4.0
communication                                  5.4
personnel_management                           3.4
information_management                         5.4
infrastructure                                 4.7
data_management                                3.4
math                                           5.8
data_analysis                  

В тестовом задании просили написать SQL-запрос, который выведет все сертификаты, для которых не заданы навыки, которые есть у владельца сертификата.

Пользователь Miguel Aubakirov, прошедший стажировку в Кабинете Кловери на позиции Python-программиста имеет следующие навыки:

In [93]:
from sqlalchemy import select, func
from sqlalchemy.orm import aliased
from database import session_factory
from models import InternsOrm, GradesOrm, ToolsOrm

import pandas as pd

i = aliased(InternsOrm, name="i")
t = aliased(ToolsOrm, name="t")

with session_factory() as session:
    stmt = (select(t)
            .join(i, i.user_role == t.user_role and i.internship == t.internship)
            .filter(i.internship == t.internship)
            .filter()
            )
    print(f"{stmt.compile(compile_kwargs={"literal_binds": True})}\n")
    result = session.execute(stmt)
    interns = result.scalars().all()
    for model in interns:
        print(model)
    # для удобства запишем результат запроса в pandas
    df = pd.read_sql(stmt, session.bind)

df.iloc[0].index

SELECT t.id, t.user_role, t.internship, t.pycaret, t.random_forest, t.anomaly_detection, t.gradient_boosting, t.cluster_analysis, t.k_fold, t.optuna, t.shap, t.tsfresh, t.sklearn_feature_selection, t.imblearn, t.streamlit, t.selenium, t.allure, t.nose, t.simpletest, t.docker, t.jira, t.adobe_xd, t.figma, t.mockplus, t.yandex_metrika, t.google_analytics, t.sketch, t.fontjoy, t.sqlalchemy, t.postgresql, t.git, t.django 
FROM cloveri.tools AS t JOIN cloveri.interns AS i ON i.user_role = t.user_role 
WHERE i.internship = t.internship



Index(['id', 'user_role', 'internship', 'pycaret', 'random_forest',
       'anomaly_detection', 'gradient_boosting', 'cluster_analysis', 'k_fold',
       'optuna', 'shap', 'tsfresh', 'sklearn_feature_selection', 'imblearn',
       'streamlit', 'selenium', 'allure', 'nose', 'simpletest', 'docker',
       'jira', 'adobe_xd', 'figma', 'mockplus', 'yandex_metrika',
       'google_analytics', 'sketch', 'fontjoy', 'sqlalchemy', 'postgresql',
       'git', 'django'],
      dtype='object')

In [85]:
df.iloc[0].dropna().index[3:]

Index(['pycaret', 'optuna', 'docker', 'jira', 'sqlalchemy', 'postgresql',
       'git', 'django'],
      dtype='object')

SELECT json_object_keys(to_json(json_populate_record(NULL::cloveri.tools, '{}'::JSON)))

In [113]:
with session_factory() as session:
    stmt = (select(t)
            .join(i, i.user_role == t.user_role and i.internship == t.internship)
            .filter(i.internship == t.internship)
            .filter(i.id == 'f1')
            )
    print(f"{stmt.compile(compile_kwargs={"literal_binds": True})}\n")

SELECT t.id, t.user_role, t.internship, t.pycaret, t.random_forest, t.anomaly_detection, t.gradient_boosting, t.cluster_analysis, t.k_fold, t.optuna, t.shap, t.tsfresh, t.sklearn_feature_selection, t.imblearn, t.streamlit, t.selenium, t.allure, t.nose, t.simpletest, t.docker, t.jira, t.adobe_xd, t.figma, t.mockplus, t.yandex_metrika, t.google_analytics, t.sketch, t.fontjoy, t.sqlalchemy, t.postgresql, t.git, t.django 
FROM cloveri.tools AS t JOIN cloveri.interns AS i ON i.user_role = t.user_role 
WHERE i.internship = t.internship AND i.id = 'f1'



In [97]:
from sqlalchemy import inspect

inst = inspect(model)
attr_names = [c_attr.key for c_attr in inst.mapper.column_attrs]
attr_names

['id',
 'user_role',
 'internship',
 'pycaret',
 'random_forest',
 'anomaly_detection',
 'gradient_boosting',
 'cluster_analysis',
 'k_fold',
 'optuna',
 'shap',
 'tsfresh',
 'sklearn_feature_selection',
 'imblearn',
 'streamlit',
 'selenium',
 'allure',
 'nose',
 'simpletest',
 'docker',
 'jira',
 'adobe_xd',
 'figma',
 'mockplus',
 'yandex_metrika',
 'google_analytics',
 'sketch',
 'fontjoy',
 'sqlalchemy',
 'postgresql',
 'git',
 'django']

In [103]:
print(inspect(model.__table__))

cloveri.tools
